In [1]:
"Set working directory"
import os
os.chdir('../')

In [2]:


import json
import numpy as np
import os
import pandas as pd


from math import exp, log
from typing import List, Tuple
from tqdm import tqdm

from socpd.objects import Object
from socpd.agent import Agent

from socpd.hypothesis_nw import PARAMS_INDIVIDUAL, \
    PARAMS_IPF_WEIGHTS
"""
Agentpy Agent Module
Content: Agent Classes

"""

"""comma module"""



class Individual(Agent):

    def setup(self):
        # linking with model #--> Agent
        
        self.random = self.model.random
        #self.status_quo = self.model.status_quo
        # Network method
        self.network = self.model.network
        
        #local variable
        self.status = False
        self.moving= False
        self.features = None
        self.influencing_profile : pd.DataFrame
        #self.influenced_score :  pd.Series
        #self.move_to_pos = None
        self.share_similar_diet :float = .0

        #call-out attribute from hypothesis/model 
        #self.env_beta = self.model._env_beta
        self.status_var = self.model.status_var
  
            # Hypothesis's params:
        self.params_self  = self.model.rules['actions_to_self']
        self.params_nw = self.model.rules['actions_to_nw']
            # for masking actions
        homo_neg  = self.model.homo_neg # neg_nw_actions
        homo_pos = self.model.homo_pos # pos_nw_actions
        
        
        #____________________ to Network _______________________
        
        # to update influencing_profile by agent's status
        self.masked_neg = np.isin(self.params_nw.index, homo_neg)
        self.masked_pos = np.isin(self.params_nw.index, homo_pos)


    @staticmethod
    def get_p(sum_betas:float):
        '''logit reversed'''
        return exp(sum_betas)/(1+exp(sum_betas))
    
    #_________________________________________________________________
    # SETUP agent's status at t = 0
    def get_status_step0(self):
        """
        Get agent status at t = 0 
        Returns: None -> change agents' status to True if status_var_yes = 1
        """
        fs = self.features
        self.status = fs[f'{self.status_var}_yes'] == 1 

    #________________________________________________________________
    # Set influencing profile by statu
    def update_influencing_profile_by_status(self):
        '''Set feature-customized influencing-profile of each agents     
        Required: 
            Hypothesis rules (param_nw)
            updated status (from t0)
            features
        Return: None
            Update influencing_profile by status 
            actions against agent's status will be zero out
        '''
        params_nw = np.array(self.params_nw)
        features = np.array(self.features)
        influencing_profile = np.multiply(params_nw,features)
        # zero-out the influence doesn't match agent's status
        if self.status:
            influencing_profile[self.masked_neg,:] = 0.0
        else:
            influencing_profile[self.masked_pos,:] = 0.0

        # set agent's influencing profile
        self.influencing_profile = influencing_profile
    
        
    def update_agent_combined(self):
        
        '''Finalize all score combination
            - update segregration
            - update self.status'''
            
        """Get scores for all actions to self"""
        #status_quo = self.model.status_quo    
        _score_self = np.array(self.params_self).dot(np.array(self.features))
        _score_self_adopt = np.sum(_score_self)
        _score_adopt = _score_self_adopt #+ self.env_beta + status_quo
        print(f'self score {_score_adopt}')
        """ Get scores from neighbor by shared similarity"""
        ln = self.network.graph.degree(self.network.positions[self])
        if ln > 0: 
            neighbors = self.network.neighbors(self)

            # Update segregration ( p of neighbors with same status ______________________________________________________
            #similar = len([n for n in neighbors if n.status == self.status])
            #self.share_similar_diet = similar / ln    
            # Get scores from neighbor' actions ________________________________________________________
                # get sum scores of influence from neighbors by homophily
            neigh_pfs = np.array([n.influencing_profile for n in neighbors])
            _neigh_scores = neigh_pfs.dot(np.array(self.features)) 
            _score_nw_adopt =  np.sum(_neigh_scores) 
            _score_adopt += _score_nw_adopt 
            
            pos_n = len([n for n in neighbors if n.status == True])
            print(ln)
            print(pos_n)
            
            if pos_n == ln or pos_n ==0:
                _score_adopt * ln
            else:
                _score_adopt += log(pos_n/(ln-pos_n))
        
        print(f'final score{_score_adopt}')
        """ UPDATE self.status """
        prob_adopt = self.get_p(_score_adopt)
        self.status = self.random.random() <= prob_adopt
    

    '''STEP 4 : taking action following  moving and status'''
    #def find_new_friends(self):
    #    self.grid.move_to(self, self.move_to_pos)
        
    def change_agent_features_by_status(self):
        if self.status:
            self.features[self.features.index.str.contains(pat = f'{self.status_var}')] = [0,1]
        else:
            self.features[self.features.index.str.contains(pat = f'{self.status_var}')] = [1,0]    



class Populating(Object):
    def __init__(self, model):
        super().__init__(model)
        self.nprandom = model.nprandom
        self.all_possible_features = self.model.all_possible_features
           
    def sampling_from_ipf(self, dir_params: str, pop:int) -> pd.DataFrame:
        """
        Sample from IPF distribution saved
        as `weights.csv` in the parameters folder

        Parameters
        ----------
        pop (int): size of data sample
        dir_params (str): path to the parameters folder

        Returns
        -------
        sample (pandas.dataFrame): dataframe containing the sampling
        """
        fpath_weights = os.path.join(dir_params, PARAMS_IPF_WEIGHTS)
        assert os.path.isfile(fpath_weights)
        df_weights = pd.read_csv(fpath_weights, sep=",", index_col=0)
        weights = df_weights["weight"] / df_weights["weight"].sum()
        indices = df_weights.index
        

        sample_indices = self.nprandom.choice(indices, pop, p=weights)
        sample = df_weights.loc[sample_indices].drop(["weight"], axis=1)
        sample = sample.reset_index(drop=True)
        return sample

    def populate_ipf(self, dir_params: str, pop:int) -> List:
        """
        Create a population of individual agents
        with the given weights obtained via IPF

        Args:
            pop (int): size of data sample.
            dir_params (str): path to parameters folder.

        Returns:
            List[Individual]: A list containing instances of
            the individual class, each representing an
            agent with specific features.
        """
        _features = pd.DataFrame()

        sample = self.sampling_from_ipf(dir_params, pop)

        # one-hot encoding
        encoded_columns = pd.get_dummies(sample).reindex(
            columns=self.all_possible_features,
            fill_value=0
        )
        _features = pd.concat([_features, encoded_columns], axis=1)
        #________________added
        cols = sorted(_features.columns)
        _features = _features[cols]

        # Add 'baseline' column filled with ones if this is not present yet
        if 'baseline' not in _features.columns:
            _features.insert(0, "baseline", 1)

        return [_features.iloc[i] for i in
                tqdm(range(pop), desc="Populating individuals", unit="i")]

    def populate_simple(self, dir_params: str, pop:int) -> List:
        """
        Create a population of individual agents
        with the given feature parameters.

        Args:
            pop (int): population size, i.e., number of agents.
            dir_params (str): dir to the folder containing
            feature parameter file.
            #from_scratch (bool, optional): flag of creating hypothesis
            from scratch or reading from files. Defaults to False.

        Returns:
            list[Individual]: a list of Individual agents
        """
        assert pop > 0, 'Size must be positive!'
        assert isinstance(pop, int), 'Pop - population size must be integer!'
        assert os.path.isdir(dir_params), "Given folder doesn't exist!"

        fpath_params_individual = os.path.join(dir_params, PARAMS_INDIVIDUAL)
        with open(fpath_params_individual) as f:
            features = json.load(f)
        features = {k.lower():v for k, v in features.items()}

        _features = pd.DataFrame()
        for feature, distribution in features.items():
            _features[feature] = self.nprandom.choice(
                distribution[0], pop, p=distribution[1]
            )

            # Define all possible columns (including those not in the sample)
            # When the sample size is too small,
            # this doesn't cover all categories,
            # the resulting DataFrame thus lacks those columns.
            # To solve the issue, we ensure all possible categories are present
            # when creating the dummy variables

        # one-hot encoding
        categorical_cols = _features.select_dtypes(include=['object'])
        encoded_cols = pd.get_dummies(categorical_cols).reindex(
            columns=self.all_possible_features,
            fill_value=0
        )
        _features.drop(categorical_cols.columns, axis=1, inplace=True)
        _features = pd.concat([_features, encoded_cols], axis=1)
        #________________added
        cols = sorted(_features.columns)
        _features = _features[cols]

        # Add 'baseline' column filled with ones
        _features.insert(0, "baseline", 1)

        return [_features.iloc[i] for i in
                tqdm(range(pop), desc="Populating individuals", unit="i")]
    



In [3]:
import numpy as np
from math import log
import networkx as nx

#agentpy
from socpd.model import Model
from socpd.network import Network
from socpd.sequences import AgentList

from socpd.hypothesis_nw import Hypothesis
 

class SocPD(Model, Hypothesis):  
    def setup(self) :
               
        # Set-up Hypothesis 
        Hypo_dict = self.p.Hypothesis_settings #_____________ added
        Hypothesis.validate_n_read_hypotheses(Hypo_dict)#__________________added

        #call-out hypothesis on actions
        self._dir_params = Hypothesis.dir_params
        self.status_var = Hypothesis.status_var
        self.all_possible_features = Hypothesis.all_possible_features
  
        self.homo_neg = Hypothesis.homo_neg
        self.homo_pos = Hypothesis.homo_pos    
        self.rules = Hypothesis.rules
        
        # Private attributes_________________________________________________________________
            # status_quo, env_beta,use_ipf, pop, 
        #self.status_quo : float = .0
        if 'env_beta' in self.p:
            self._env_beta = float(self.p['env_beta'])
        else:
            self._env_beta = 0.0
        self.report("intervention's effect-env_beta", self._env_beta)

            # Specifying use ipf
        #self._use_ipf : bool = None
        if 'use_ipf' in self.p:
            self._use_ipf = self.p['use_ipf']
        else:
            self._use_ipf = False 
        self.report('use_ipf', self._use_ipf)
                
        #----------------------------------------------#
        #_______PREPARE NETWORK_____________________

        pop = self.pop = self.p['pop']
        m = self.p['m']
        p = self.p['p']
        q = self.p['q']
        graph = nx.extended_barabasi_albert_graph(
            n = pop, 
            m = m, # there are 1-p-q chance a new nodes can connect with m other existing nodes based on attachment preference
            p = p, # each existing nodes has p prob to form a new links to the others with attachment preference
            q = q, # each existing nodes has q prob to rewire one of thier existin links with attachment preference
            seed=self.random)
        self.network = Network(self, graph)
        
            # Report network's attributes
        degree_sequence = np.array([d for _, d in graph.degree()])
        self.report("Max_nw_size", np.max(degree_sequence))
        self.report("Min_nw_size", np.min(degree_sequence))
        self.report("AVG_nw_size", np.mean(degree_sequence))
        
        
        #_______GENERATE AGENTS______________________
        self.agents = AgentList(self, pop, Individual)
        
            # generate features then update agent's features and status
            # Specifying use_ipf
        self.Populating = Populating(self)
        if self._use_ipf:
            _feature_iter = self.Populating.populate_ipf(self._dir_params, pop)
        else:
            _feature_iter = self.Populating.populate_simple(self._dir_params, pop)
            
            # update agent's features and status step 0
        for i, a in enumerate(self.agents):
            a.features = _feature_iter[i]
            
        self.agents.get_status_step0()
        
        #_______ADD AGENTS ON network_________________________
        
        self.network.add_agents(self.agents, self.network.nodes)

  
    def update(self): 
        positive_p = len(self.agents.select(self.agents.status==True))/self.pop
                # Stop simulation if all are positive or negative
        if positive_p <= 0.01 or positive_p >= 0.99:
            self.stop()
        
        #self.status_quo = log(positive_p/(1-positive_p))

        #record status
        self['positive'] = positive_p
        self.record('positive')
        self['negative'] = 1 - self['positive']
        self.record('negative')
        
        # update from t = 0 
        self.agents.update_influencing_profile_by_status()  
        #self.agents.update_status_quo()
        
        #if self.nw3D:
        self.agents.update_agent_combined()
        #self.unhappy = self.agents.select(self.agents.moving == True)
        #self.unhappy.find_new_friends()
     
 


       
    def step(self) : 

        self.agents.change_agent_features_by_status()
        
    #def get_segregation(self):
        # Calculate average percentage of similar neighbors
    #    return round(sum(self.agents.share_similar_diet) / self.pop, 2)

    def end(self):
        # Measure segregation at the end of the simulation
        #self.report('segregation', self.get_segregation())        
        self.report(f'Final_{self.status_var}_proportion', self.positive) 
        self.report(f'Peak_{self.status_var}_proportion', max(self.log['positive']))








In [4]:
dir_params = 'parameters_demo'
all_possible_actions=  [ 'Veg_prob',
                          'concerns_health',
                         'concerns_animal_welfare',
                         'concerns_environement',
                         'concerns_convinience',
                         'concerns_familiarity',
                         'concerns_taste',
                         'concerns_price',
                         'concerns_hunger',
                         'homo_veg_inf',
                         'homo_om_inf',

                        ]

Hypothesis_settings = { 'dir_params' : dir_params , # folder name stored actions parameters
                        'status_var':'vegetarian',  # the variables deciding agents' status
                        # all actions from the actions parameters
                        'all_possible_actions':all_possible_actions,
                        # all actions/events influencing directly on the agent
                        'actions_to_self':['Veg_prob',
                                            'concerns_health',
                                            'concerns_animal_welfare',
                                            'concerns_environement',
                                            'concerns_convinience',
                                            'concerns_familiarity',
                                            'concerns_taste',
                                            'concerns_price',
                                            'concerns_hunger',],
                                            
                        # all actions/events from the agent influencing on others in thier networks
                        'actions_to_nw':['homo_veg_inf',
                                        'homo_om_inf',],

                        # actions/events from the agents having negative effects on thier networks 
                        'actions_to_nw_neg':['homo_om_inf'],
                        # actions/events from the agents having positive effects on thier networks 
                        'actions_to_nw_pos':['homo_veg_inf',],
                        }
 


parameters = {'Hypothesis_settings' : Hypothesis_settings,
            'steps': 4,    
            'seed' : 42,   
            'pop' : 20, # size of the population
            'm': 1, # Number of edges with which a new node attaches to existing nodes
            'p' : 0, # each existing nodes has p prob to form m new link(s) to the others with attachment preference
            'q' : .78 # each existing nodes has q prob to rewire m existing link(s) with attachment preference
            #'use_ipf': False,   
            } 


In [5]:
model = SocPD(parameters)
result= model.run()

Populating individuals: 100%|██████████| 20/20 [00:00<00:00, 7183.26i/s]

self score -1.7190000000000003
5
0
final score-11.827000000000002
self score 2.513
2
0
final score-0.2370000000000001
self score -0.6440000000000002
2
0
final score-3.9780000000000006
self score -1.0259999999999994
4
0
final score-5.511999999999999
self score 0.8439999999999998
1
0
final score-1.7730000000000001
self score -0.26600000000000046
1
0
final score-1.9280000000000004
self score 0.5939999999999998
5
0
final score-9.701
self score -0.2960000000000004
2
0
final score-4.23
self score -0.29899999999999893
3
0
final score-6.164999999999998
self score -3.452000000000001
2
0
final score-8.386000000000001
self score 2.716
1
0
final score1.1710000000000003
self score -2.0540000000000003
1
0
final score-0.7540000000000002
self score -1.1390000000000005
2
0
final score-2.431
self score -1.6320000000000001
1
0
final score-3.5940000000000003
self score -1.186
2
0
final score-1.548
self score 2.4739999999999993
final score2.4739999999999993
self score 1.7640000000000002
1
0
final score0.68